In [39]:
import tensorflow as tf
import numpy as np

x_data = [[1, 2, 1, 1],
          [2, 1, 3, 2],
          [3, 1, 3, 4],
          [4, 1, 5, 5],
          [1, 7, 5, 5],
          [1, 2, 5, 6],
          [1, 6, 6, 6],
          [1, 7, 7, 7]]
# 8 * 4
y_data = [[0, 0, 1],
          [0, 0, 1],
          [0, 0, 1],
          [0, 1, 0],
          [0, 1, 0],
          [0, 1, 0],
          [1, 0, 0],
          [1, 0, 0]]

# 입력 데이터를 ndarray로 변환하지만, 이미 ndarray일 경우에는
# 메모리에 새로 ndarray를 생성하지는 않는다.
x_data = np.asarray(x_data, dtype=np.float32)
y_data = np.asarray(y_data, dtype=np.float32)
nb_classes = 3

# 먼저 Weight와 bias setting
W = tf.Variable(tf.random.normal([4, nb_classes]), name=
                'weight')
# 4 * 3
b = tf.Variable(tf.random.normal([nb_classes]), name='bias')
variables = [W, b]

variables

[<tf.Variable 'weight:0' shape=(4, 3) dtype=float32, numpy=
 array([[-0.62458473,  0.4234396 , -1.0987784 ],
        [ 0.20714556,  1.6756382 , -0.45219132],
        [ 0.42504662,  0.68134964, -1.1231694 ],
        [-0.7173501 ,  1.0393274 , -0.6222199 ]], dtype=float32)>,
 <tf.Variable 'bias:0' shape=(3,) dtype=float32, numpy=array([ 0.48150456, -0.9141008 , -0.06468647], dtype=float32)>]

In [40]:
def hypothesis(X):
    return tf.nn.softmax(tf.matmul(X, W) + b)
# 8 * 3
print(hypothesis(x_data))

tf.Tensor(
[[9.9261124e-03 9.8985004e-01 2.2380595e-04]
 [1.5761345e-03 9.9842179e-01 2.1263938e-06]
 [1.6492640e-05 9.9998355e-01 1.6750594e-08]
 [5.9787271e-07 9.9999940e-01 1.8791416e-11]
 [2.0678201e-09 1.0000000e+00 5.1594750e-15]
 [5.5130062e-07 9.9999940e-01 4.0881559e-11]
 [1.1996237e-09 1.0000000e+00 1.3533690e-15]
 [3.6902870e-11 1.0000000e+00 5.0352014e-18]], shape=(8, 3), dtype=float32)


In [41]:
# 점검
sample_db = [[8, 2, 1, 4]]
sample_db = np.array(sample_db, dtype=np.float32)

print(hypothesis(sample_db))

tf.Tensor([[3.3605112e-08 1.0000000e+00 3.6463898e-11]], shape=(1, 3), dtype=float32)


In [42]:
def cost_fn(X, Y):
    cost = -tf.reduce_sum(Y * tf.math.log(hypothesis(X)), axis=1)
    # axis=1이면 행단위로 sum
    cost_mean = tf.reduce_mean(cost)
    
    return cost_mean

print(cost_fn(x_data, y_data))

tf.Tensor(10.49183, shape=(), dtype=float32)


In [43]:
x = tf.constant(3.0)
# tf.Tensor
with tf.GradientTape() as g:
    g.watch(x)
    y = x*x
dy_dx = g.gradient(y, x)
print(dy_dx)

tf.Tensor(6.0, shape=(), dtype=float32)


In [44]:
#Gradient 계산
def grad_fn(X, Y):
    with tf.GradientTape() as tape:
        loss = cost_fn(X, Y)
        grads = tape.gradient(loss, variables)
        
        return grads
print(grad_fn(x_data, y_data))

[<tf.Tensor: id=301515, shape=(4, 3), dtype=float32, numpy=
array([[-0.24835865,  0.9983302 , -0.7499715 ],
       [-1.6223192 ,  2.122263  , -0.4999438 ],
       [-1.6231613 ,  2.4981325 , -0.8749713 ],
       [-1.6233561 ,  2.4983277 , -0.8749715 ]], dtype=float32)>, <tf.Tensor: id=301514, shape=(3,), dtype=float32, numpy=array([-0.24856001,  0.6235318 , -0.37497175], dtype=float32)>]


In [45]:
def fit(X, Y, epochs = 2000, verbose=100):
    optimizer = tf.keras.optimizers.SGD(learning_rate = 0.1)
    
    for i in range(epochs):
        grads = grad_fn(X, Y)
        optimizer.apply_gradients(zip(grads, variables))
        # variables = [W, b]
        if (i==0) | (i+1)%verbose==0:
            print('Loss at epoch %d : %f' %(i+1, cost_fn(X, Y).numpy()))
            
fit(x_data, y_data)

Loss at epoch 100 : 0.611678
Loss at epoch 200 : 0.528341
Loss at epoch 300 : 0.476533
Loss at epoch 400 : 0.434171
Loss at epoch 500 : 0.396086
Loss at epoch 600 : 0.359963
Loss at epoch 700 : 0.324296
Loss at epoch 800 : 0.288052
Loss at epoch 900 : 0.252435
Loss at epoch 1000 : 0.231716
Loss at epoch 1100 : 0.220287
Loss at epoch 1200 : 0.209932
Loss at epoch 1300 : 0.200481
Loss at epoch 1400 : 0.191820
Loss at epoch 1500 : 0.183854
Loss at epoch 1600 : 0.176504
Loss at epoch 1700 : 0.169701
Loss at epoch 1800 : 0.163387
Loss at epoch 1900 : 0.157512
Loss at epoch 2000 : 0.152032


## Prediction Check

In [46]:
sample_data = [[2, 1, 3, 2]]
sample_data = np.asarray(sample_data, dtype=np.float32)

a = hypothesis(sample_data)

print(a)
print(tf.argmax(a, 1))
# tf.argmax는 가장 큰 값의 인덱스를 반환. 1은 axis

tf.Tensor([[0.00180109 0.07805449 0.92014444]], shape=(1, 3), dtype=float32)
tf.Tensor([2], shape=(1,), dtype=int64)


In [47]:
b = hypothesis(x_data)
print(b)
print(tf.argmax(b, 1))
print(tf.argmax(y_data, 1))
#맞는 것도 있고, 틀린 것도 있고..

tf.Tensor(
[[5.19746209e-06 1.35833642e-03 9.98636425e-01]
 [1.80109218e-03 7.80545101e-02 9.20144439e-01]
 [2.10684927e-08 1.57926470e-01 8.42073500e-01]
 [4.18982722e-07 8.56208861e-01 1.43790722e-01]
 [2.50895321e-01 7.37412333e-01 1.16923684e-02]
 [1.31969541e-01 8.67921710e-01 1.08736625e-04]
 [7.61733532e-01 2.38210127e-01 5.63513750e-05]
 [9.17484820e-01 8.25140029e-02 1.12362295e-06]], shape=(8, 3), dtype=float32)
tf.Tensor([2 2 2 1 1 1 0 0], shape=(8,), dtype=int64)
tf.Tensor([2 2 2 1 1 1 0 0], shape=(8,), dtype=int64)
